In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import json
import httpx
import string
import secrets

from time import sleep

from getpass import getpass

from app.models import DeploymentContext
from app.auth import token_to_payload

<IPython.core.display.Javascript object>

In [4]:
base_url = "http://localhost:8000"
username = "jochen"
password = getpass()
service = "single_cast_hosting"

# get access_token
client = httpx.Client(base_url=base_url)
r = client.post("/token", data={"username": username, "password": password})
access_token = r.json()["access_token"]
headers={"authorization": f"Bearer {access_token}"}

# get service token
data = {"service": service, "origin": "notebook"}
r = client.post("/service-token", json=data, headers=headers)
service_token = r.json()["service_token"]
headers={"authorization": f"Bearer {service_token}"}
client.headers = headers

 ········


<IPython.core.display.Javascript object>

# Deploy

In [5]:
def show_details(details_url):
    r = client.get(details_url, headers=headers)
    rdata = r.json()
    len_steps = len([s for s in rdata["steps"] if s["state"] == "success"])
    started = rdata["started"]
    finished = rdata["finished"]
    print(f"{len_steps} {started} {finished}")

<IPython.core.display.Javascript object>

In [6]:
alphabet = string.ascii_letters + string.digits
database_password = ''.join(secrets.choice(alphabet) for i in range(20))
secret_key = "".join(secrets.choice(alphabet) for i in range(32))

<IPython.core.display.Javascript object>

In [14]:
fqdn = "foobar.staging.wersdoerfer.de"
env = {
    "fqdn": fqdn,
    "database_name": f"cast_{fqdn}",
    "database_user": f"cast_{fqdn}",
    "database_password": database_password,
    "secret_key": secret_key,
    "port": 10001,
    "settings_file_name": fqdn.replace(".", "_")
}
context = DeploymentContext(env=env)
r = client.post("/deployments/", json=context.dict(), headers=headers)
rdata = r.json()
print(rdata)
details_url = r.json()["details"]

# for i in range(3):
#     show_details(details_url)
#     sleep(1)

{'id': 125, 'service_id': 6, 'origin': 'notebook', 'user': 'jochen', 'started': '2022-01-23T21:35:55.711897', 'finished': None, 'context': {'env': {'fqdn': 'foobar.staging.wersdoerfer.de', 'database_name': 'cast_foobar.staging.wersdoerfer.de', 'database_user': 'cast_foobar.staging.wersdoerfer.de', 'database_password': '6It0n1QGoMM4GtAkGmRr', 'secret_key': '2oCTxJIo9YBpWquiVG5Ks3cJHcIPKZUB', 'port': 10001, 'settings_file_name': 'foobar_staging_wersdoerfer_de'}}, 'type': 'deployment', 'deleted': False, 'details': 'http://localhost:8000/deployments/125'}


<IPython.core.display.Javascript object>

In [1]:
foo = {"bar": "baz"}

In [2]:
dict(foo, blub="bla")

{'bar': 'baz', 'blub': 'bla'}